
### Objective: 

In this assignment, implement the Node2Vec algorithm, a random-walk-based GNN, to learn node embeddings. Train a classifier using the learned embeddings to predict node labels.

### Dataset: 

Cora dataset: The dataset consists of 2,708 nodes (scientific publications) with 5,429 edges (citations between publications). Each node has a feature vector of size 1,433, and there are 7 classes (research topics).
Skeleton Code:

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from node2vec import Node2Vec  # Importing Node2Vec for the random walk

# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop

def train():
    for epoch in range(100):
        classifier.train()
        optimizer.zero_grad()
        
        # Get node embeddings as input
        output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
        
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train()

print("Training complete!")


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:13<00:00,  7.27it/s]


Epoch 0, Loss: 1.9608745574951172
Epoch 10, Loss: 1.2374218702316284
Epoch 20, Loss: 0.8905270099639893
Epoch 30, Loss: 0.7458568811416626
Epoch 40, Loss: 0.6765410304069519
Epoch 50, Loss: 0.6362488865852356
Epoch 60, Loss: 0.6089761257171631
Epoch 70, Loss: 0.5889682769775391
Epoch 80, Loss: 0.5732861161231995
Epoch 90, Loss: 0.5605018734931946
Training complete!


## Explanation:
Node2Vec generates node embeddings by simulating random walks on the graph. These walks capture structural properties of nodes.
The generated embeddings are then used to train a classifier for predicting node labels.
The Cora dataset is a benchmark graph where nodes are papers and edges are citations.

## Questions (1 point each):
1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?
2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

4. What would happen if we used directed edges instead of undirected edges for the random walks?
5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?
6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?
8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?



### Extra credit: 
1. What would happen if we increased the window size (window) for the skip-gram model? How would it affect the embedding quality?

## No points, just for you to think about
7. What would happen if we removed self-loops from the graph before training Node2Vec?

9. What would happen if we applied normalization to the node embeddings before feeding them to the classifier?

### 1. What would happen if we increased the number of walks (num_walks) per node? How might this affect the learned embeddings?

In [4]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=500, workers=2)
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

train()

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 250/250 [00:46<00:00,  5.34it/s]


Epoch 0, Loss: 2.0586254596710205
Epoch 10, Loss: 1.2902240753173828
Epoch 20, Loss: 0.9253613948822021
Epoch 30, Loss: 0.7644137740135193
Epoch 40, Loss: 0.6871922612190247
Epoch 50, Loss: 0.6434270739555359
Epoch 60, Loss: 0.6147224307060242
Epoch 70, Loss: 0.5941963791847229
Epoch 80, Loss: 0.5785537958145142
Epoch 90, Loss: 0.56610506772995
Training complete!


Increasing the number of walks in Node2Vec provides more training data by exploring more paths in the graph, leading to more robust and detailed node embeddings. This should result in faster convergence and lower training loss, as the model captures the graph's structure more effectively. We can see that this does in fact lead to a lower training loss.

### 2. What would happen if we reduced the walk length (walk_length)? How would this influence the structural information captured by the embeddings?

In [5]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=10, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop

def train():
    for epoch in range(100):
        classifier.train()
        optimizer.zero_grad()
        
        # Get node embeddings as input
        output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
        
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train()

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 100/100 [00:04<00:00, 23.67it/s]


Epoch 0, Loss: 2.02813458442688
Epoch 10, Loss: 1.2759928703308105
Epoch 20, Loss: 0.9090166091918945
Epoch 30, Loss: 0.7447628974914551
Epoch 40, Loss: 0.6635621190071106
Epoch 50, Loss: 0.6174379587173462
Epoch 60, Loss: 0.5875948071479797
Epoch 70, Loss: 0.5660608410835266
Epoch 80, Loss: 0.549502968788147
Epoch 90, Loss: 0.5361846089363098
Training complete!


Reducing the walk length in Node2Vec limits the exploration range of each random walk, causing the embeddings to focus more on the local neighborhood of each node. This would lead to less comprehensive structural information, making the embeddings more biased toward capturing close connections rather than the broader graph structure.

### 4. What would happen if we used directed edges instead of undirected edges for the random walks?

In [6]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=False)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=2) 
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(64, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop

def train():
    for epoch in range(100):
        classifier.train()
        optimizer.zero_grad()
        
        # Get node embeddings as input
        output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
        
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train()

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:14<00:00,  6.96it/s]


Epoch 0, Loss: 1.9939035177230835
Epoch 10, Loss: 1.294539213180542
Epoch 20, Loss: 0.9360954761505127
Epoch 30, Loss: 0.7753670811653137
Epoch 40, Loss: 0.6957798004150391
Epoch 50, Loss: 0.6492462158203125
Epoch 60, Loss: 0.6185664534568787
Epoch 70, Loss: 0.5967667102813721
Epoch 80, Loss: 0.580217719078064
Epoch 90, Loss: 0.566978394985199
Training complete!


Using directed edges in Node2Vec means that the random walks respect the direction of the edges, capturing the asymmetric relationships between nodes. This should lead to embeddings that reflect the influence of directionality in the graph.

### 5. What would happen if we added more features to the nodes (e.g., 2000-dimensional features instead of 1433)?

Adding more features to the nodes provides the model with a deeper understanding of each node, potentially capturing more complex patterns and relationships in the data. However, this also increases the risk of overfitting, especially if the additional features are not informative or if the dataset is small. If the data isn't as complex as the model it would lead to overfitting.

### 6. What would happen if we used a different dataset with more classes? Would the classifier performance change significantly?

Using a dataset with more classes increases the complexity of the classification task, requiring the model to learn more nuanced differences between the classes. This often leads to lower initial performance, as the model needs to capture a broader range of features, but it can improve with sufficient data and training to handle the increased diversity.

### 8. What would happen if we used a larger embedding dimension (e.g., 128 instead of 64)? How would this affect the model’s performance and training time?

In [8]:
# Load the Cora dataset
dataset = Planetoid(root='data/Cora', name='Cora')

# Prepare data
data = dataset[0]

# Convert to networkx for random walk
import networkx as nx
G = to_networkx(data, to_undirected=True)

# Node2Vec configuration
node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=2)
model = node2vec.fit(window=10, min_count=1)

# Embeddings for each node
embeddings = model.wv  # Node embeddings

# Define a simple classifier
class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Classifier, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# Initialize classifier and optimizer
classifier = Classifier(128, 7)
optimizer = optim.Adam(classifier.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# Training loop

def train():
    for epoch in range(100):
        classifier.train()
        optimizer.zero_grad()
        
        # Get node embeddings as input
        output = classifier(torch.tensor([embeddings[str(i)] for i in range(data.num_nodes)]))
        
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train()

print("Training complete!")

Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 2): 100%|██████████| 100/100 [00:14<00:00,  7.13it/s]


Epoch 0, Loss: 1.9860632419586182
Epoch 10, Loss: 1.1065382957458496
Epoch 20, Loss: 0.7770000696182251
Epoch 30, Loss: 0.659434974193573
Epoch 40, Loss: 0.5973513126373291
Epoch 50, Loss: 0.5561709403991699
Epoch 60, Loss: 0.5259265899658203
Epoch 70, Loss: 0.502881646156311
Epoch 80, Loss: 0.4844331443309784
Epoch 90, Loss: 0.46912264823913574
Training complete!


Increasing the embedding dimension to 128 allows the model to capture more detailed and complex patterns in the graph, leading to improved performance as it learns richer representations. However, this also increases the training time and computational cost due to the larger size of the embeddings.